In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train.shape

(60000, 28, 28)

In [117]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

batch_size=5000

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    #tf.set_random_seed(2)
    
    X = tf.placeholder(dtype=tf.float32)
    X2 = tf.placeholder(dtype=tf.float32)
    Y = tf.placeholder(dtype=tf.float32)
    
    num_hidden=256


    # Build a hidden layer Left
    W_hidden = tf.Variable(tf.random.normal([784, num_hidden]))
    b_hidden = tf.Variable(tf.random.normal([num_hidden]))
    p_hidden = tf.nn.relu( tf.add(tf.matmul(X, W_hidden), b_hidden) )

#    W_hidden2 = tf.Variable(tf.random.normal([num_hidden, num_hidden]))
#    b_hidden2 = tf.Variable(tf.random.normal([num_hidden]))
#    p_hidden2 = tf.nn.relu( tf.add(tf.matmul(p_hidden, W_hidden2), b_hidden2) )

    # Build a hidden layer Right
    W_hiddenR = tf.Variable(tf.random.normal([784, num_hidden]))
    b_hiddenR = tf.Variable(tf.random.normal([num_hidden]))
    p_hiddenR = tf.nn.relu( tf.add(tf.matmul(X2, W_hiddenR), b_hiddenR) )

#    W_hidden2R = tf.Variable(tf.random.normal([num_hidden, num_hidden]))
#    b_hidden2R = tf.Variable(tf.random.normal([num_hidden]))
#    p_hidden2R = tf.nn.relu( tf.add(tf.matmul(p_hiddenR, W_hidden2R), b_hidden2R) )

    # Conncatenate Left + Right
    W_concat = tf.Variable(tf.random.normal([num_hidden,num_hidden]))
    b_concat = tf.Variable(tf.random.normal([num_hidden]))
    p_concat2 = tf.add(tf.matmul(tf.divide(tf.math.add(p_hidden,p_hiddenR),2), W_concat), b_concat)
    p_concat3 = tf.layers.batch_normalization(tf.reshape(p_concat2,[batch_size,num_hidden]),momentum=0.9)
    p_concat22=tf.nn.relu(p_concat3)
    
    # Build the output layer
    W_output = tf.Variable(tf.random.normal([num_hidden, 10]))
    b_output = tf.Variable(tf.random.normal([10]))
    p_output = tf.nn.softmax( tf.add(tf.matmul(p_concat22, W_output), b_output))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=p_output))
    acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(Y,1), 
                                      predictions=tf.argmax(p_output,1))
    
    minimization_op = tf.train.GradientDescentOptimizer(learning_rate=0.008).minimize(loss)
    saver = tf.train.Saver()
    
    limit=10000
    
    def norm(x):
        return (x-np.min(x))/(np.max(x)-np.min(x))

    feed_dict = {
        X: norm(x_train.reshape(-1,784)),
        X2: norm(x_train.reshape(-1,784)),
        Y: pd.get_dummies(y_train)
    }

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [119]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle).astype(np.float32), np.asarray(labels_shuffle).astype(np.float32)

In [120]:

with tf.Session(graph=graph, config=config) as session:
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    session.run(init)
    
    for step in range(5000):

        X0, Y0 = next_batch(batch_size, x_train, y_train)
        X02=X0
        
        feed_dict = {
        X: norm(X0.reshape(-1,784)),
        X2: norm(X02.reshape(-1,784)),
        Y: pd.get_dummies(Y0)}
        
        J_value = session.run(loss, feed_dict)
        acc = (batch_size-session.run(tf.count_nonzero(tf.add(tf.argmax(session.run(Y, feed_dict),1),-tf.argmax(session.run(p_output, feed_dict),1)))))/batch_size
        if step % 10 == 0:
            print("Step:", step, " Loss:", J_value," Accuracy:", acc)
        session.run(minimization_op, feed_dict)


Step: 0  Loss: 2.361038  Accuracy: 0.1
Step: 10  Loss: 2.3641238  Accuracy: 0.097
Step: 20  Loss: 2.3691053  Accuracy: 0.092
Step: 30  Loss: 2.374696  Accuracy: 0.086
Step: 40  Loss: 2.3718145  Accuracy: 0.089
Step: 50  Loss: 2.3680196  Accuracy: 0.093
Step: 60  Loss: 2.3811338  Accuracy: 0.08
Step: 70  Loss: 2.3679104  Accuracy: 0.093
Step: 80  Loss: 2.357213  Accuracy: 0.104
Step: 90  Loss: 2.3560512  Accuracy: 0.105
Step: 100  Loss: 2.3692105  Accuracy: 0.092


KeyboardInterrupt: 

In [ ]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    W_hidden_value = session.run(W_hidden)
    print("Hidden layer weight matrix:\n", W_hidden_value)
    b_hidden_value = session.run(b_hidden)
    print("Hidden layer bias:\n", b_hidden_value)
    W_output_value = session.run(W_output)
    print("Output layer weight matrix:\n", W_output_value)
    b_output_value = session.run(b_output)
    print("Output layer bias:\n", b_output_value)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, np.argmax(np.array(pred00).reshape(60000,-1),axis=1))